In [1]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense, StringLookup
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.src.layers import LSTM

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)
product_data['color_mapped'] = pd.factorize(product_data['productColorName'])[0]

pandas_interaction_data = pd.merge(pandas_interaction_data, product_data[['main_product_id', 'color_mapped', 'productName']],
                                   left_on='main_product_id',
                                   right_on='main_product_id',
                                    how='left')

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount,color_mapped,productName
0,4589682,3692870,1,3,Schnürstiefelette Glattleder braun
1,10133267,8476659,1,3,PG8002 Sneaker high Rauleder braun
2,10148602,4125038,1,6,Schnürstiefelette Glattleder beige
3,7806599,3504372,1,4,Sportliche Ballerina Rauleder grau
4,6480452,3591163,1,0,Sneaker high Glattleder schwarz
...,...,...,...,...,...
995,8651989,3591582,1,6,Sportliche Ballerina Rauleder beige
996,7506341,3504775,1,3,Chelsea Boot Rauleder braun
997,9242957,7830955,1,38,Sandaleno met sleehak Combi leer/Imitatieleer ...
998,6336427,3696847,1,2,Sneaker low Glattleder weiß


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40....",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]",30
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]",56
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[],1
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0],33


In [4]:
# Initialize tokenizer
string_lookup = StringLookup()
string_lookup.adapt(pandas_interaction_data['productName'])

tokenizer = Tokenizer(num_words=string_lookup.vocabulary_size(), oov_token="<OOV>")
tokenizer.fit_on_texts(pandas_interaction_data['productName'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(pandas_interaction_data['productName'])

# Pad sequences
padded_sequences = pad_sequences(sequences, padding='post')

In [5]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values, 
    'product_color': pandas_interaction_data['color_mapped'].values, 
    'product_description': padded_sequences, 
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [6]:
interaction_dataset

<_TensorSliceDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(8,), dtype=tf.int32, name=None)}>

In [7]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(8,), dtype=tf.int32, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(8,), dtype=tf.int32, name=None)}>

In [8]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

unique_colors = np.array(product_data["color_mapped"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)

product_colors_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_colors_vocabulary.adapt(unique_colors)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [9]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, unique_product_color_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])

        # Color embeddings
        self.color_embedding = tf.keras.Sequential([
            unique_product_color_vocab,
            Embedding(unique_product_color_vocab.vocabulary_size(), embedding_dimension)
        ]) 


        # Textual description model
        self.textual_description_model = tf.keras.Sequential([
            Embedding(input_dim=unique_product_ids_vocab.vocabulary_size(), output_dim=embedding_dimension),
            LSTM(64),
            Dense(64, activation='relu')
        ])
        
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, activation="relu")

        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])

        # New embeddings for color and description
        color_embeddings = self.color_embedding(features["product_color"])
        description_embeddings = self.textual_description_model(features["product_description"])
    
        # Combine existing and new embeddings
        # Note: You might have to adjust the axis depending on your data shapes
        user_combined = tf.concat([user_embeddings], axis=1)
        product_combined = tf.concat([product_embeddings, color_embeddings, description_embeddings], axis=1)


        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [10]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, product_colors_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [11]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3) 
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
7/7 [==============================] - 3s 210ms/step - auc: 0.0000e+00 - rmse: 1.0001 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0150 - factorized_top_k/top_5_categorical_accuracy: 0.0425 - factorized_top_k/top_10_categorical_accuracy: 0.0675 - factorized_top_k/top_50_categorical_accuracy: 0.1800 - factorized_top_k/top_100_categorical_accuracy: 0.2775 - loss: 493.5761 - regularization_loss: 0.0000e+00 - total_loss: 493.5761
Epoch 2/3
7/7 [==============================] - 1s 199ms/step - auc: 0.0000e+00 - rmse: 0.9793 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.2750 - factorized_top_k/top_5_categorical_accuracy: 0.4487 - factorized_top_k/top_10_categorical_accuracy: 0.5175 - factorized_top_k/top_50_categorical_accuracy: 0.7300 - factorized_top_k/top_100_categorical_accuracy: 0.8300 - loss: 491.5901 - regularization_loss: 0.0000e+00 - total_loss: 491.5901
Epoch 3/3
7/7 [=======

In [12]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

7/7 [==============================] - 2s 216ms/step - auc: 0.0000e+00 - rmse: 0.9299 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9638 - factorized_top_k/top_5_categorical_accuracy: 0.9837 - factorized_top_k/top_10_categorical_accuracy: 0.9887 - factorized_top_k/top_50_categorical_accuracy: 0.9950 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 487.9295 - regularization_loss: 0.0000e+00 - total_loss: 487.9295


([0.0,
  0.929881751537323,
  0.0,
  0.0,
  0.9637500047683716,
  0.9837499856948853,
  0.9887499809265137,
  0.9950000047683716,
  1.0,
  109.63053894042969,
  0,
  109.63053894042969],
 [0.0, 0.0, 0.0])

In [13]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

4/4 [==============================] - 1s 216ms/step - auc: 0.0000e+00 - rmse: 0.9857 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.1300 - factorized_top_k/top_5_categorical_accuracy: 0.2750 - factorized_top_k/top_10_categorical_accuracy: 0.3550 - factorized_top_k/top_50_categorical_accuracy: 0.5300 - factorized_top_k/top_100_categorical_accuracy: 0.6250 - loss: 166.3991 - regularization_loss: 0.0000e+00 - total_loss: 166.3991


([0.0,
  0.985679566860199,
  0.0,
  0.0,
  0.12999999523162842,
  0.2750000059604645,
  0.35499998927116394,
  0.5299999713897705,
  0.625,
  16.669906616210938,
  0,
  16.669906616210938],
 [0.0, 0.0, 0.0])

In [14]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		0.985679566860199
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		0.12999999523162842
factorized_top_k/top_5_categorical_accuracy		0.2750000059604645
factorized_top_k/top_10_categorical_accuracy		0.35499998927116394
factorized_top_k/top_50_categorical_accuracy		0.5299999713897705
factorized_top_k/top_100_categorical_accuracy		0.625
AUC: 0.0


In [15]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 5181267,  6264129,  1982475,  9441144,  4668062,  5086233,
        6704019,  9966299,  5872343,  2225186,  5739629, 10762597,
        1991139,  7744656,  2167927,  5810378,  4375062,  9149416,
        6162861,  7364737,  7233329,  9807258,  2016930,  5628821,
        7552534,  1985973,  9069465,  9801605,  6954165,  6159706,
       10464464,  2075624,  6156578,  5550099,  1958367,  6200210,
        8805246,  4254525, 10845833,  1999453,  7727094,  8384842,
       10715540,  5616577,  2076462, 10804908,  9227165,  9762573,
        6564829,  8924125, 10179673,  6906362,  8691401,  8826570,
        8541944,  3856322,  9688791, 10180721,  9338707,  8324291,
        4328308,  2004061,  6567105, 11073231])

In [16]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  5181267
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
101,3504775,4062862281702,Chelsea Boot Rauleder braun,braun,56.661.32,6590678,315582,w,PT,H-W,...,40.5,7.0,12.0,11.0,TR,0.0,0.0,"[3504775, 3477007, 3504771, 3478521, 3478174, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
300,6115810,4064032440902,Chelsea Boot Rauleder,,72.091.47,6590678,315582,w,SK,H-W,...,39.0,6.0,10.0,11.0,TR,1.0,1.0,"[6115810, 6163657, 6460667, 6249160, 6282880, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",15
590,3846282,4062862149972,Eleganter Pumps Rauleder schwarz,schwarz,52.151.47,6590678,315573,w,SK,H-W,...,38.5,5.5,9.0,0.0,TPU,1.0,1.0,"[3846282, 3504440, 3850372, 4771268, 3850054, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
622,4771321,4062862905509,Plateau Pumps Effektleder silber,silber,61.260.61,6590677,315569,w,PT,F-S,...,35.5,3.0,4.0,0.0,EVA,0.0,1.0,"[4771321, 4771326, 4771325, 4771327, 4771322, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",14
720,3850944,4062862925996,Eleganter Pumps Rauleder rosa,rosa,55.380.14,6590677,315573,w,PT,H-W,...,35.0,2.5,3.0,0.0,TPU,0.0,1.0,"[3850944, 4846948, 4846936, 5184972, 5184966, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",8
795,4770125,4062862810346,Pantolette Rauleder schwarz,schwarz,66.743.47,6590678,315572,w,PT,F-S,...,38.0,5.0,8.0,0.0,TR,0.0,1.0,"[4770125, 5336309, 5336297, 4770124, 5336298, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
1218,6113376,4064032454244,Biker- / Combat Boot Rauleder braun,braun,72.704.41,6590678,5014852,w,PT,H-W,...,39.0,6.0,10.0,18.0,TR,0.0,0.0,"[6113376, 6116055, 6116347, 6113377, 6113382, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
1326,6114331,4064032421734,Sportlicher Stiefel Glattleder braun,braun,72.799.24,6590678,5014850,w,SK,H-W,...,36.0,3.5,5.0,42.0,TR,1.0,1.0,"[6114331, 6112111, 6639919, 6112110, 6639915, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
1811,7592808,4065171055422,Eleganter Ballerina Glattleder schwarz,schwarz,81.301.30,6590677,315576,w,PT,F-S,...,39.0,6.0,10.0,0.0,TPU,0.0,0.0,"[7592808, 7592804, 7592806, 7592811, 7592805, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",0
1860,7589216,4064032401354,Riemchensandale Glattleder rosa,rosa,81.710.20,6590677,315563,w,PT,F-S,...,43.0,9.0,16.0,0.0,TPU,0.0,0.0,"[7589216, 7589204, 7589215, 7589209, 7589213, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",8
1951,7800773,4064032948538,Slip-on Suède beige,beige,84.213.14,6590677,315576,w,PT,F-S,...,43.0,9.0,16.0,0.0,TPU,1.0,1.0,"[7800773, 7799248, 7800435, 8531254, 8531265, ...","[37.0, 38.0, 39.0, 40.0, 40.5, 41.0, 43.0, 44.0]",6


scores:  [0.02741655 0.02274698 0.02273702 0.02264103 0.02191615 0.02052831
 0.02041268 0.01939073 0.01890537 0.01858329]


In [17]:
user_id = next(iter(train.take(1)))['customer_id'].numpy()
predict_user(user_id)

predicting user:  9785974
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
1022,5386334,4062862555599,Slipper Glattleder grau,grau,62.449.61,6590678,315576,w,PT,F-S,...,37.0,4.0,6.0,0.0,EVA,0.0,0.0,"[5386334, 5386338, 5386341, 5386339, 5386333, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",4


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
154,3693255,4062862067450,Schnürstiefelette Glattleder braun,braun,51.642.20,6590677,315587,w,PT,H-W,...,40.0,6.5,11.0,16.0,EVA,0.0,0.0,"[3693255, 3711711, 3692820, 3711704, 3692825, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
188,3696977,4062862931959,Chelsea Boot Glattleder rot,rot,54.670.55,6590677,315582,w,PT,H-W,...,36.0,3.5,5.0,14.0,TR,0.0,0.0,"[3696977, 3696983, 3743546, 3743541, 3696988, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",5
216,3743199,4062862288329,Plateau Stiefelette Glattleder schwarz,schwarz,55.771.27,6590677,315595,w,PT,H-W,...,44.0,9.5,17.0,11.0,TPU,0.0,0.0,"[3743199, 3800405, 3788343, 3788347, 3743194, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
251,3693104,4062862092650,Schnürstiefelette Rauleder blau,blau,0873.90.11,3692257,315587,w,PT,H-W,...,42.0,8.0,14.0,14.0,TR,0.0,0.0,"[3693104, 3693106, 3843334, 3693107, 3693103, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
417,4031030,4062862029663,Sneaker low Glattleder braun,braun,53.492.25,6590677,315567,w,PT,H-W,...,40.5,7.0,12.0,0.0,PU-TPU,0.0,0.0,"[4031030, 4031029, 4031028, 4031027, 4031026, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
575,4846759,4062862518082,Keilsandalette Rauleder braun,braun,64.642.18,6590677,315564,w,PT,F-S,...,36.0,3.5,5.0,0.0,TR,0.0,0.0,"[4846759, 4772166, 4775821, 4775817, 4772179, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
674,3835430,4062862669852,Eleganter Pumps Lederimitat rosa,rosa,51.400.74,6590677,315573,w,PT,H-W,...,40.5,7.0,12.0,0.0,TPU,1.0,1.0,"[3835430, 3834613, 3834610, 4772475, 4772476, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",8
693,4771778,4062862871309,Sneaker low Materialmix Leder/Lederimitat weiß,weiß,66.946.51,4997827,315567,w,VN,F-S,...,40.5,7.0,12.0,0.0,Gummi-EVA,0.0,1.0,"[4771778, 4771777, 4771776, 4774059, 4774070, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",2
963,5337018,4064032098356,Plateau Sandale Rauleder grün,grün,64.661.19,6590677,315568,w,SK,F-S,...,36.0,3.5,5.0,0.0,PU-TPU,0.0,0.0,"[5337018, 5337019, 5337024, 5338595, 5338589, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",11
1652,7233592,4064032948323,Winterstiefelette Lackleder schwarz,schwarz,71.720.70,6590677,315580,w,PT,H-W,...,39.0,6.0,10.0,15.0,PU-TPU,0.0,0.0,"[7233592, 7233591, 7233593, 7233588, 7233594, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",0


scores:  [0.027441   0.02647044 0.02423124 0.02345612 0.02302378 0.02164039
 0.02124399 0.02070319 0.02057302 0.02007197]


In [18]:
model.save('../../models/gabor/gabor_tensorflow_reco_all_data_3_epochs_v1.h5')

/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.